# 🐦 EfficientNetB2 Bird Species Classifier

This notebook prepares a bird species image dataset, builds an image classification model using EfficientNetB2, and trains the model using PyTorch.

In [ ]:
import sys
import os
!{sys.executable} -m pip install aiohttp aiofiles pandas requests matplotlib torch torchvision torchinfo tqdm ipywidgets
sys.path.append(os.path.abspath(".."))

In [ ]:
import asyncio
import json
import random
import shutil
import time
from pathlib import Path

import aiohttp
import pandas as pd
import numpy as np
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from config.config import (
	TEXAS_BIRDS_CSV,
	TAXA_URL,
	PAGES_TO_FETCH,
	DATASET_PATH,
	RAW_DATA_PATH,
	MODEL_SAVE_PATH,
	RESULTS_SAVE_PATH,
	BATCH_SIZE,
	NUM_WORKERS,
	SEED,
	TRAIN_SPLIT_RATIO,
	NUM_CLASSES,
	LEARNING_RATE,
	EPOCHS,
	EARLY_STOPPING_PATIENCE,
	EARLY_STOPPING_DELTA,
)

from scripts.fetch_taxon_id import get_taxon_id_by_name
from scripts.fetch_images import process_species
from scripts.rename_files import rename_image_files


In [ ]:
birds_df = pd.read_csv(TEXAS_BIRDS_CSV)
taxon_ids = []

for bird in birds_df["bird_name"]:
  print(f"Fetching taxon_id for '{bird}'")
  taxon_id = get_taxon_id_by_name(TAXA_URL, bird)
  taxon_ids.append(taxon_id)
  time.sleep(1) # 1 second sleep to follow API rate limits

birds_df.insert(1, column = "taxon_id", value = taxon_ids)
birds_df.to_csv(TEXAS_BIRDS_CSV, index=False)

In [ ]:
async def download_images():
  all_licensing = []
  global_semaphore = asyncio.Semaphore(1)

  async with aiohttp.ClientSession() as session:
    tasks = []
    for index, row in birds_df.iterrows():
      bird_name = row["bird_name"]
      taxon_id = row["taxon_id"]
      tasks.append(process_species(session, bird_name, taxon_id, PAGES_TO_FETCH, RAW_DATA_PATH, global_semaphore))
    results = await asyncio.gather(*tasks)
    for metadata in results:
      all_licensing.extend(metadata)

  licensing_csv = os.path.join(RAW_DATA_PATH, "attribution_metadata.csv")
  pd.DataFrame(all_licensing).to_csv(licensing_csv, index=False)
  print(f"Licensing metadata saved to {licensing_csv}")

In [ ]:
await download_images()

In [ ]:
def rename_image_files_all_folders(directory: str) -> None:
	for root, dirs, _ in os.walk(directory):
		for d in dirs:
			folder_path = os.path.join(root, d)
			print(f"🔍 Processing: {folder_path}")
			rename_image_files(folder_path)

In [ ]:
rename_image_files_all_folders(DATASET_PATH)

# splitting images into train and test folders:

In [ ]:
def train_test_split(original_data_dir: str, output_base_dir: str, train_ratio: float = 0.8):
	"""
	Splits a dataset of images organized by class folders into training and testing sets.
	Each subdirectory in the original dataset represents a class (e.g., a bird species),
	and contains image files. This function randomly splits each class's images into
	training and testing subsets according to the given ratio, and copies them into
	corresponding 'train' and 'test' folders under the output directory.
	Args:
		original_data_dir (str): Path to the original dataset directory. Must contain
															subdirectories for each class/species.
		output_base_dir (str): Path to the base directory where the 'train' and 'test'
														folders will be created.
		train_ratio (float, optional): Proportion of images to use for training.
																		Remaining images will be used for testing.
																		Defaults to 0.8.
	Raises:
		AssertionError: If original_data_dir is not a valid directory.
	Returns:
		None
	"""
	assert os.path.isdir(original_data_dir), f"{original_data_dir} is not a valid directory"

	species_folders = [f for f in os.listdir(original_data_dir) if os.path.isdir(os.path.join(original_data_dir, f))]

	for species in species_folders:
		species_path = os.path.join(original_data_dir, species)
		images = [f for f in os.listdir(species_path)
							if os.path.isfile(os.path.join(species_path, f))
							and not (f.startswith('.') or f == "DS_Store")]

		random.shuffle(images)
		train_count = int(len(images) * train_ratio)
		train_images = images[:train_count]
		test_images = images[train_count:]

		# output species directories
		train_species_dir = os.path.join(output_base_dir, "train", species)
		test_species_dir = os.path.join(output_base_dir, "test", species)
		os.makedirs(train_species_dir, exist_ok=True)
		os.makedirs(test_species_dir, exist_ok=True)

		# copy files into new directories
		for img in train_images:
			shutil.copy2(os.path.join(species_path, img), os.path.join(train_species_dir, img))
		for img in test_images:
			shutil.copy2(os.path.join(species_path, img), os.path.join(test_species_dir, img))

		print(f"Done splitting {species}: {len(train_images)} train / {len(test_images)} test")

In [ ]:
train_test_split(RAW_DATA_PATH, DATASET_PATH, TRAIN_SPLIT_RATIO)

In [ ]:
rename_image_files_all_folders(DATASET_PATH)

In [4]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

mps


In [ ]:
def create_effnetb2_model(num_classes:int, seed:int=42):
	"""
	Creates a pretrained EfficientNetB2 model adapted for custom image classification,
	along with appropriate training and validation transforms.
	This function:
	- Loads EfficientNetB2 with ImageNet weights.
	- Replaces the final classification layer to match the number of target classes.
	- Returns a model ready for fine-tuning (all layers are trainable).
	- Provides data augmentation transforms for training.
	- Provides ImageNet-style transforms for validation.
	Args
			num_classes (int, optional): Number of output classes for classification.
			seed (int, optional): Random seed for reproducible initialization of
														the final classifier layer. Defaults to 42.
	Returns:
			Tuple[
					torch.nn.Module,          # The EfficientNetB2 model with modified classifier
					torchvision.transforms.Compose,  # Transformations for training data
					torchvision.transforms.Compose   # Transformations for validation data
			]
  """
	weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT

	train_transforms = transforms.Compose([
	transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
	transforms.RandomHorizontalFlip(p=0.5),
	transforms.RandomRotation(degrees=15),
	transforms.ToTensor(),
	transforms.Normalize(mean=[0.485, 0.456, 0.406],
												std=[0.229, 0.224, 0.225])
	])

	val_transforms = weights.transforms()
	model = torchvision.models.efficientnet_b2(weights=weights)

	for p in model.features.parameters():
		p.requires_grad = True

	torch.manual_seed(seed)
	model.classifier = nn.Sequential(
		nn.Dropout(p=0.3, inplace=True),
		nn.Linear(in_features=1408, out_features=num_classes)
	)

	return model, train_transforms, val_transforms

In [ ]:
effnetb2, train_tf, val_tf = create_effnetb2_model(num_classes=NUM_CLASSES, seed=SEED)

In [7]:
effnetb2 = effnetb2.to(device)

In [ ]:
from torchinfo import summary

summary(effnetb2, 
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 35]              --                   True
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   True
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   True
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    864                  True
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    64                   True
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 16, 112,

In [ ]:
train_dir = Path(DATASET_PATH) / "train"
test_dir = Path(DATASET_PATH) / "test"

In [ ]:
def create_dataloaders(
  train_dir: str, 
  test_dir: str, 
  train_transform: transforms.Compose,
  val_transform:  transforms.Compose,
  batch_size: int, 
  num_workers: int = NUM_WORKERS
):
  train_ds = datasets.ImageFolder(train_dir, transform=train_transform)
  val_ds = datasets.ImageFolder(test_dir, transform=val_transform)
  class_names = train_ds.classes

  train_dataloader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=False
  )
  test_dataloader = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=False
  )

  return train_dataloader, test_dataloader, class_names

In [ ]:
train_dataloader_effnetb2, test_dataloader_effnetb2, class_names = create_dataloaders(train_dir=train_dir,
                                                                                    test_dir=test_dir,
                                                                                    train_transform=train_tf,
                                                                                    val_transform=val_tf,
                                                                                    batch_size=BATCH_SIZE)

In [ ]:
class EarlyStopping:
	"""
	Simple early‐stopper that checkpoints the best model and
	stops when val_loss hasn’t improved for `patience` epochs.
	"""
	def __init__(self, patience:int=3, delta:float=0.0, path:str="best_model.pth"):
		"""
		Args:
			patience: how many epochs to wait after last time val_loss improved
			delta: minimum change in val_loss to qualify as an improvement
			path: where to save the best model weights
		"""
		self.patience = patience
		self.delta = delta
		self.path = path

		self.best_loss = np.inf
		self.num_bad_epochs = 0

	def __call__(self, val_loss:float, model:torch.nn.Module):
		# save and reset counter if loss improved by at least delta 
		if val_loss + self.delta < self.best_loss:
			self.best_loss = val_loss
			self.num_bad_epochs = 0
			torch.save(model.state_dict(), self.path)
			print(f"  ↳ val_loss improved to {val_loss:.4f}, saving model.")
		else:
			self.num_bad_epochs += 1
			print(f"  ↳ no improvement ({self.num_bad_epochs}/{self.patience})")

		# return True if we’ve hit patience
		return self.num_bad_epochs >= self.patience


In [ ]:
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
	
  model.train()
  train_loss, train_acc = 0, 0
  for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      train_loss += loss.item() 
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  model.eval() 
  test_loss, test_acc = 0, 0
  with torch.inference_mode():
      for batch, (X, y) in enumerate(dataloader):
          X, y = X.to(device), y.to(device)
          test_pred_logits = model(X)
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()
          test_pred_labels = test_pred_logits.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          early_stopper: EarlyStopping,
          device: torch.device) -> Dict[str, List]:
    results = {"train_loss": [],
                "train_acc": [],
                "test_loss": [],
                "test_acc": []}
    for epoch in range(1, epochs+1):
        # 1) train + test
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
        test_loss, test_acc   = test_step(model, test_dataloader, loss_fn, device)

        # 2) log
        print(f"Epoch {epoch}/{epochs} | "
                f"train_loss: {train_loss:.4f} | train_acc: {train_acc:.4f} | "
                f"test_loss: {test_loss:.4f} | test_acc: {test_acc:.4f}")

        # 3) record history
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        # 4) early stop?
        if early_stopper is not None:
            stop = early_stopper(test_loss, model)
            if stop:
                print(f"Stopped early at epoch {epoch}")
                break

    # restore best weights if we used early stopping
    if early_stopper is not None:
        model.load_state_dict(torch.load(early_stopper.path))
    return results

In [ ]:
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
	"""
	Performs one training epoch on the given dataloader.

	Args:
		model (nn.Module): The PyTorch model to train.
		dataloader (DataLoader): DataLoader for the training dataset.
		loss_fn (nn.Module): Loss function to use.
		optimizer (Optimizer): Optimizer for updating model weights.
		device (torch.device): Device to perform computations on (e.g., 'cuda' or 'cpu').

	Returns:
		Tuple[float, float]: Average training loss and accuracy for the epoch.
	"""
	model.train()
	train_loss, train_acc = 0.0, 0.0

	for batch, (X, y) in enumerate(dataloader):
		X, y = X.to(device), y.to(device)
		y_pred = model(X)
		loss = loss_fn(y_pred, y)
		train_loss += loss.item()

		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

		y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
		train_acc += (y_pred_class == y).sum().item() / len(y_pred)

	train_loss /= len(dataloader)
	train_acc /= len(dataloader)
	return train_loss, train_acc


def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
	"""
	Evaluates the model on the test/validation dataset.

	Args:
		model (nn.Module): The PyTorch model to evaluate.
		dataloader (DataLoader): DataLoader for the test/validation dataset.
		loss_fn (nn.Module): Loss function to use.
		device (torch.device): Device to perform computations on.

	Returns:
		Tuple[float, float]: Average test loss and accuracy for the epoch.
	"""
	model.eval()
	test_loss, test_acc = 0.0, 0.0

	with torch.inference_mode():
		for batch, (X, y) in enumerate(dataloader):
			X, y = X.to(device), y.to(device)
			test_pred_logits = model(X)
			loss = loss_fn(test_pred_logits, y)
			test_loss += loss.item()

			test_pred_labels = test_pred_logits.argmax(dim=1)
			test_acc += (test_pred_labels == y).sum().item() / len(test_pred_labels)

	test_loss /= len(dataloader)
	test_acc /= len(dataloader)
	return test_loss, test_acc


def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          early_stopper: EarlyStopping,
          device: torch.device) -> Dict[str, List]:
	"""
	Trains and evaluates the model over multiple epochs, with optional early stopping.
	Args:
		model (nn.Module): The model to train.
		train_dataloader (DataLoader): Dataloader for training data.
		test_dataloader (DataLoader): Dataloader for test/validation data.
		optimizer (Optimizer): Optimizer for training.
		loss_fn (nn.Module): Loss function to optimize.
		epochs (int): Number of epochs to train.
		early_stopper: EarlyStopping object to monitor validation loss and stop early if needed.
		device (torch.device): Device to use for training and evaluation.
	Returns:
		Dict[str, List[float]]: Dictionary containing lists of training/test loss and accuracy per epoch.
	"""
	results = {
		"train_loss": [],
		"train_acc": [],
		"test_loss": [],
		"test_acc": []
	}

	for epoch in range(1, epochs + 1):
		# 1: train and evaluate
		train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
		test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)

		# 2: log metrics
		print(f"Epoch {epoch}/{epochs} | "
					f"train_loss: {train_loss:.4f} | train_acc: {train_acc:.4f} | "
					f"test_loss: {test_loss:.4f} | test_acc: {test_acc:.4f}")

		# 3: save metrics
		results["train_loss"].append(train_loss)
		results["train_acc"].append(train_acc)
		results["test_loss"].append(test_loss)
		results["test_acc"].append(test_acc)

		# 4: early stopping check
		if early_stopper is not None:
			stop = early_stopper(test_loss, model)
			if stop:
				print(f"Early stopping at epoch {epoch}")
				break
                  
	# best model
	if early_stopper is not None:
		model.load_state_dict(torch.load(early_stopper.path))

	return results

In [ ]:
optimizer_finetune = torch.optim.Adam(params=effnetb2.parameters(), lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()
stopper = EarlyStopping(
	patience=EARLY_STOPPING_PATIENCE,
	delta=EARLY_STOPPING_DELTA,
	path=MODEL_SAVE_PATH
)

# model training
effnetb2_results = train(
    model=effnetb2,
    train_dataloader=train_dataloader_effnetb2,
    test_dataloader=test_dataloader_effnetb2,
    epochs=EPOCHS,
    optimizer=optimizer_finetune,
    loss_fn=loss_fn,
    device=device,
    early_stopper=stopper
)


Epoch 1/10 | train_loss: 1.2877 | train_acc: 0.6714 | test_loss: 0.3332 | test_acc: 0.9041
  ↳ val_loss improved to 0.3332, saving model.
Epoch 2/10 | train_loss: 0.4425 | train_acc: 0.8668 | test_loss: 0.2447 | test_acc: 0.9214
  ↳ val_loss improved to 0.2447, saving model.
Epoch 3/10 | train_loss: 0.3034 | train_acc: 0.9073 | test_loss: 0.2071 | test_acc: 0.9319
  ↳ val_loss improved to 0.2071, saving model.
Epoch 4/10 | train_loss: 0.2247 | train_acc: 0.9301 | test_loss: 0.2122 | test_acc: 0.9296
  ↳ no improvement (1/3)
Epoch 5/10 | train_loss: 0.1714 | train_acc: 0.9466 | test_loss: 0.1976 | test_acc: 0.9377
  ↳ val_loss improved to 0.1976, saving model.
Epoch 6/10 | train_loss: 0.1417 | train_acc: 0.9552 | test_loss: 0.1857 | test_acc: 0.9390
  ↳ val_loss improved to 0.1857, saving model.
Epoch 7/10 | train_loss: 0.1132 | train_acc: 0.9646 | test_loss: 0.1837 | test_acc: 0.9430
  ↳ val_loss improved to 0.1837, saving model.
Epoch 8/10 | train_loss: 0.1000 | train_acc: 0.9680 | te

In [ ]:
# save training results
os.makedirs("../models", exist_ok=True)
with open(RESULTS_SAVE_PATH, "w") as f:
    json.dump(effnetb2_results, f)